In [48]:
# 1. Pricing a Barrier Option
# --> GBM 이용한 option pricing

# LN 7주차 : Generating Sample Paths
# LN 12주차 : Sampling processes with jump
# HW#2-3 (bsmopm)
# HW#3-1 (barrier option pricing with discretization scheme)
# HW#3-3-(a) (Euler discretization을 이용한 sample paths 부분)

import numpy as np
from scipy import stats

# params. set
S_0 = 100
L = 105.
T = 1.
r = 0.05 # RN 적용으로 mu 대체
sig = 0.25

np.random.seed(0)
N = int(1e6) # no. of sample paths

Z_T_over2 = np.random.randn(N)*np.sqrt(0.5*T) # T/2시점의 변동성
S_T_over2 = S_0*np.exp((r-0.5*sig*sig)*(0.5*T) + sig*Z_T_over2) # T/2시점의 Stock price

Z_T = np.random.randn(N)*np.sqrt(T) # T시점의 변동성
S_T = S_T_over2*np.exp((r-0.5*sig*sig)*(T) + sig*Z_T) # T/2 시점의 Stock price(S_T_over2)를 베이스로 다시 T/2 이후 시점의 Stock price (S_T) 계산(step-by-step)

# digital option은 아니지만, T/2시점의 stock price 조건에 따라 pv 추정이 달라짐을 고려
H_0_1st_cond = np.exp(-r*T)*(S_T_over2 <= L) # T/2시점의 stock price가 L 이하일 경우
H_0_2nd_cond = np.exp(-r*T)*(S_T_over2 > L)  # T/2시점의 stock price가 L 초과일 경우

# 1st 조건의 경우, option price의 pv가 0보다 작은경우 하방위험은 없으므로 max 조건에 의해 pay-off는 0 적용
print('Estimated H_0 price with 1st_cond. (95% CI) = ', np.mean(H_0_1st_cond*(S_T-110)), ' ± ', 1.96*np.std(H_0_1st_cond*(S_T-110))/np.sqrt(N))
print('Estimated H_0 price with 2nd_cond. (95% CI) = ', np.mean(H_0_2nd_cond*(S_T-120)), ' ± ', 1.96*np.std(H_0_2nd_cond*(S_T-120))/np.sqrt(N))

Estimated H_0 price with 1st_cond. (95% CI) =  -8.37493681571483  ±  0.04009749297724383
Estimated H_0 price with 2nd_cond. (95% CI) =  2.3731730637946544  ±  0.04197988143245037


In [56]:
# 2. Pricing a Barrier Option with Jumps
# --> 1번과 컨셉 동일 + JD application

# LN 11주차 : Jump-diffusion Process 중 Poisson process ~ Jump diffusion model 
# LN 12주차 : Sampling processes with jump 제일 마지막 장 !!
# 1203_Sampling Process with Jump.ipynb + Option Pricing with Jump.py 참조

import numpy as np
from scipy import stats

# params. set
S_0 = 100
L = 105.
T = 1.
r = 0.05 # RN 적용으로 mu 대체
sig = 0.25

# jump-term params. set
avg_jump_num = 2.0
avg_jump_logsize = 0.0
vol_jump_logsize = 0.2

# Risk-neutral valuation 보정계수 eta
eta = np.exp(avg_jump_logsize + vol_jump_logsize**2/2)-1

np.random.seed(0)
N = int(1e6) # no. of sample paths

#Z_T = np.random.normal(loc=0.0, scale=np.sqrt(T), size=N)
jump_num = np.random.poisson(lam=avg_jump_num*T, size=N)
jump_size = np.ones(N)

for i in range(N):
    if(jump_num[i]>0):
        jump_size[i] = np.prod(np.exp(np.random.normal(loc=avg_jump_logsize, scale=vol_jump_logsize, size=jump_num[i])))
        
Z_T_over2 = np.random.normal(loc=0.0, scale=np.sqrt(0.5*T), size=N) # T/2시점의 변동성
S_T_over2_w_jump = S_0*np.exp((r-avg_jump_num*eta-0.5*sig*sig)*(0.5*T) + sig*Z_T_over2)*jump_size # T/2시점의 stock price with jump-term (보정계수 -lam*eta 적용)

Z_T = np.random.normal(loc=0.0, scale=np.sqrt(T), size=N) # T시점의 변동성
S_T_w_jump = S_T_over2_w_jump*np.exp((r-avg_jump_num*eta-0.5*sig*sig)*(T) + sig*Z_T) # T/2 시점의 Stock price(S_T_over2)를 베이스로 다시 T/2 이후 시점의 Stock price (S_T) 계산(step-by-step)

# 1번과 동일하게 "digital option은 아니지만, T/2시점의 stock price 조건에 따라 pv 추정이 달라짐을 고려"
H_0_1st_cond_w_jump = np.exp(-r*T)*(S_T_over2_w_jump <= L) # T/2시점의 stock price가 L 이하일 경우
H_0_2nd_cond_w_jump = np.exp(-r*T)*(S_T_over2_w_jump > L)  # T/2시점의 stock price가 L 초과일 경우

# 1st 조건의 경우, option price의 pv가 0보다 작은경우 하방위험은 없으므로 max 조건에 의해 pay-off는 0 적용
# -> jump-term이 option pricing에 적용됨에 따라 1번결과와 비교했을 때 위아래 쌍방향으로의 변동성이 더욱 커짐을 확인
print('Estimated H_0 with jump price with 1st_cond. (95% CI) = ', np.mean(H_0_1st_cond_w_jump*(S_T_w_jump-110)), ' ± ', 1.96*np.std(H_0_1st_cond_w_jump*(S_T_w_jump-110))/np.sqrt(N))
print('Estimated H_0 with jump price with 2nd_cond. (95% CI) = ', np.mean(H_0_2nd_cond_w_jump*(S_T_w_jump-120)), ' ± ', 1.96*np.std(H_0_2nd_cond_w_jump*(S_T_w_jump-120))/np.sqrt(N))

Estimated H_0 with jump price with 1st_cond. (95% CI) =  -15.340060707286648  ±  0.04566199013735011
Estimated H_0 with jump price with 2nd_cond. (95% CI) =  7.200590424944668  ±  0.06201166312008933


In [74]:
# 3. Basket Call Option Pricing
# --> using Multivariate-Normal
# cf) Asian Call : 시간 축 상(time-series) 평균값 correlation 
# --> 변형해서 각 portfolio를 담은 산술평균에서 strike 뺀 max값 산출 (서로 다른 Brownian Motion이 공분산을 갖는 cross-sectional correlation)

# LN 7주차 : Generating sample paths 중 "Sampling BM from Multivariate-Normal" 
# HW#2-5 Asian option pricing by simulation 변형해서 basket call로 적용해보기

import numpy as np

np.random.seed(0)
N = int(1e6) # no. of sample paths

K = 100
T = 5
r = 0.05

M = 5
dt = T/M

Z_vec = np.zeros((N,M+1)) 
t = np.zeros(M+1)     

sig_vec = np.zeros(M+1)
sig_vec = [0, 0.1, 0.15, 0.2, 0.25, 0.3]

S0_vec = np.zeros(M+1)
S0_vec = [0, 90, 95, 100, 105, 110]

mu_vec = np.zeros(M+1)
mu_vec = [0, 0.05, 0.06, 0.07, 0.08, 0.09]

Sig_matrix = np.zeros((M+1,M+1))

for i in range(M+1):
    for j in range(M+1):
        
        # correlation structure의 rho (60%) 적용
        if(i>=j):
            Sig_matrix[i,j] = 0.6*j*dt
        else:
            Sig_matrix[i,j] = 0.6*i*dt
        
Z = np.random.multivariate_normal(mu_vec, Sig_matrix, N)

S_T_vec = np.zeros((N,M+1)) # T시점 Basket별 price

for i in range(M):
    t[i+1] = t[i] + dt
    S_T_vec[:,i+1] = S0_vec[i+1]*np.exp((r-sig_vec[i+1]*sig_vec[i+1]/2)*t[i+1]+sig_vec[i+1]*Z[:,i+1]) 
    
S_T_bar = np.zeros(N)  # Basket별 산술 평균

for k in range(N):
    S_T_bar[k] = np.mean(S_T_vec[k,:])
    
# Call option pay-off 및 PV estimation
C_T = (S_T_bar-K)*(S_T_bar>K)
C_0 = np.exp(-r*T)*C_T
C_0_e = np.mean(C_0)
C_0_std = np.std(C_0)

print("C_0 = ", C_0_e)
print("95% CI = ", [C_0_e-1.96*C_0_std/np.sqrt(N), C_0_e+1.96*C_0_std/np.sqrt(N)])

C_0 =  6.722304947290939
95% CI =  [6.694824383067402, 6.749785511514476]


In [89]:
# 4. Option Pricing under a Stochastic Volatility Model
# --> volatility term이 변동

# LN 7주차 : Generating sample paths 중 Discretization Methods ~ Euler scheme
# LN 8-9주차 : Financial Modeling Examples(I) 중 Implied Volatility ~ Stochastic Volatility model(Heston)

# BSM_examle_BHK_Euler.py 참조
# Euler discretization 부분
# HW#3-3-(a) Barrier option under CEV model 참조

import numpy as np
from scipy import stats

# params. set
S_0 = 100.
K = 100.
T = 1.0
r = 0.05
sig = 0.3

np.random.seed(0)

N = int(1e6)
M = 250
dt = T/M

k = 3
theta = 0.05
V_0 = 0.04

V_T = np.ones(N)*V_0

# Euler discretization이 적용된 volatility term
for j in range(M):
    W = np.random.randn(N)
    V_T = V_T + k*(theta-V_T)*dt + sig*np.sqrt(V_T*(V_T>0))*W*np.sqrt(dt)

S_T = np.ones(N)*S_0

# Euler discretization이 적용된 T시점의 Stock price
for k in range(M):
    Z = np.random.randn(N)
    S_T = S_T + r*S_T*dt + V_T*S_T*Z*np.sqrt(dt)

# Option price의 PV
C_0 = np.exp(-r*T)*(S_T-K)*(S_T>K)

print('Estimated Call Price (95% CI) = ',np.mean(C_0),' ± ',1.96*np.std(C_0)/np.sqrt(N))

Estimated Call Price (95% CI) =  5.425819546634073  ±  0.009259377908614402


In [92]:
# 5. Option Pricing under Stochastic interest-rate model
# --> interest-term이 변동

# LN 8-9주차 : Financial Modeling Examples(I) 중 Interest rate models ~ sampling from diffusion(III) w/ Vasicek Model
# LN 11주차 : Jump-diffusion Process 중 Poisson process ~ Jump diffusion model ~ Jump diffusion Short-rate model

import numpy as np
from scipy import stats

# params. set
S_0 = 100.
K = 100.
T = 1.0
sig = 0.2

np.random.seed(0)

N = int(1e6)
M = 250
dt = T/M

a = 3
b = 0.03
c = 0.25
R_0 = 0.05

R_T = np.ones(N)*R_0

# Euler discretization이 적용된 interest-rate term
for j in range(M):
    W = np.random.randn(N)
    R_T = R_T + a*(b-R_T)*dt + c*np.sqrt(R_T*(R_T>0))*W*np.sqrt(dt)

S_T = np.ones(N)*S_0
R_sum = 0

# Euler discretization이 적용된 T시점의 interest-rate sum 및 Stock price
for k in range(M):
    Z = np.random.randn(N)
    R_sum = R_sum + R_T*dt # 이자율 변화부분에 대한 직사각형 넓이 벡터
    S_T = S_T + R_T*S_T*dt + sig*S_T*Z*np.sqrt(dt)

# T시점의 call option price
C_T = (S_T-K)*(S_T>K)

# Call option price의 PV with risk-neutral valuation
C_0 = np.exp(-R_sum)*C_T

print('Estimated Call Price (95% CI) = ',np.mean(C_0),' ± ',1.96*np.std(C_0)/np.sqrt(N))

Estimated Call Price (95% CI) =  9.82445724276443  ±  0.028768968175020816
